### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},"{'env': {'game': 'Assault', 'type': 'atari', '...",1bescnbb
1,{'_wandb': {'runtime': 1}},{},31v22138
2,"{'act_f1': 0.13352366218750464, 'act_acc': 0.4...","{'env': {'game': 'Alien', 'type': 'atari', 'fr...",8c31f8r6
3,{},"{'env': {'game': 'Alien', 'type': 'atari', 'fr...",1p9xxn3k
4,{},"{'env': {'game': 'Alien', 'type': 'atari', 'fr...",2a9rf0va
...,...,...,...
719,"{'negative_sim': 0.04744461342226714, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
720,"{'_step': 117001, '_wandb': {'runtime': 18252}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
721,"{'negative_sim': 0.18186782717704772, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
722,"{'positive_sim': 0.9873969799280168, 'loss': -...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline_10'
exp_name = 'bc'
model_path = '0/100/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

20


/tmp/ipykernel_8637/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
41,"{'act_acc': 0.2793156307190657, '_runtime': 33...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",31ktobwm,Krull
42,"{'_step': 223000, 'epoch': 90, 'reward_f1': 0....","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",z7un794r,Kangaroo
43,"{'act_f1': 0.39769741833559713, '_timestamp': ...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",1vg8yjnp,KungFuMaster
44,"{'best_metric_val': 0.4311041672550959, 'epoch...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",p1gfgr7u,MsPacman
49,"{'act_acc': 0.43298441044668806, '_runtime': 3...","{'env': {'game': 'Jamesbond', 'type': 'atari',...",4ushilzk,Jamesbond
50,"{'reward_ratio': 0.030301515075753783, 'min_gr...","{'env': {'game': 'Frostbite', 'type': 'atari',...",3u3osku0,Frostbite
51,"{'loss': 1.4006055178200094, '_step': 234400, ...","{'env': {'game': 'Gopher', 'type': 'atari', 'f...",1rzyqidw,Gopher
52,"{'epoch': 100, 'act_f1': 0.3277654334078916, '...","{'env': {'game': 'Hero', 'type': 'atari', 'fra...",nonskx95,Hero
61,"{'_runtime': 35575, '_timestamp': 1666432920, ...","{'env': {'game': 'CrazyClimber', 'type': 'atar...",3w1wq49g,CrazyClimber
62,"{'mean_grad_norm': 0.12148486369086388, 'best_...","{'env': {'game': 'Freeway', 'type': 'atari', '...",22m8imw2,Freeway


### Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

Krull 31ktobwm
Kangaroo z7un794r
KungFuMaster 1vg8yjnp
MsPacman p1gfgr7u
Jamesbond 4ushilzk
Frostbite 3u3osku0
Gopher 1rzyqidw
Hero nonskx95
CrazyClimber 3w1wq49g
Freeway 22m8imw2
DemonAttack 3n0ts35g
ChopperCommand 1eyiz23m
BankHeist 197tid2b
BattleZone 354qqb55
Breakout b5i2o4qv
Boxing 3g1rxnht
Alien 3dycl85a
Asterix lgi6gcks
Amidar 1m20nhb9
Assault 3txfymh8


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### Generate json file

In [17]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)